# Zadania do wykonania na danych w Pandas

1. **Załaduj DataFrame do Pandas, posortuj według `timestamp`, zmień indeks na kolumnę `timestamp` oraz usuń kolumnę `index`.**
   - *Wskazówka*: Użyj `pd.read_csv()` do załadowania danych oraz `sort_values()`,`set_index()` dla ustawienia indeksu na `timestamp`.

In [73]:
import pandas as pd
pomiary = pd.read_csv('pomiary.csv')
pomiary_by_time = pomiary.sort_values(by = 'timestamp')
pomiary_by_time.head()

,index,timestamp,voltage,current,temperature,technician_first_name,technician_last_name,status
38,39,2023-01-01 05:43:00,231.267702,1.245972,101.748627,Emily,Brown,broken
211,212,2023-01-01 10:21:00,240.932880,1.527754,65.513886,Michael,Johnson,working
162,163,2023-01-01 10:25:00,240.383591,1.778813,57.968485,Emily,Brown,working
292,293,2023-01-01 12:28:00,0.000000,0.000000,54.056483,John,Doe,turned off
272,273,2023-01-01 13:59:00,240.955130,1.225599,61.739875,John,Doe,working


In [74]:
pomiary_time_index = pomiary_by_time.set_index('timestamp')
pomiary_time_index = pomiary_time_index.drop('index',axis = 1)
pomiary_time_index.head()

,voltage,current,temperature,technician_first_name,technician_last_name,status
timestamp,,,,,,
2023-01-01 05:43:00,231.267702,1.245972,101.748627,Emily,Brown,broken
2023-01-01 10:21:00,240.932880,1.527754,65.513886,Michael,Johnson,working
2023-01-01 10:25:00,240.383591,1.778813,57.968485,Emily,Brown,working
2023-01-01 12:28:00,0.000000,0.000000,54.056483,John,Doe,turned off
2023-01-01 13:59:00,240.955130,1.225599,61.739875,John,Doe,working


2. **Połącz kolumny `technician_first_name` i `technician_last_name` w jedną kolumnę o nazwie `technician`.**


In [75]:
pomiary_time_index['technician'] = pomiary_time_index['technician_first_name'] + ' ' + pomiary_time_index['technician_last_name']
pomiary_time_index = pomiary_time_index.drop(['technician_first_name','technician_last_name'],axis = 1)
print(pomiary_time_index[['technician']].head())



                          technician
timestamp                           
2023-01-01 05:43:00      Emily Brown
2023-01-01 10:21:00  Michael Johnson
2023-01-01 10:25:00      Emily Brown
2023-01-01 12:28:00         John Doe
2023-01-01 13:59:00         John Doe


3. **Przekształć kolumnę `status` na trzy nowe kolumny kategoryczne: `status_working`, `status_broken`, i `status_turned off`.**
   - *Wskazówka*: Skorzystaj z `pd.get_dummies()` lub przypisz wartości `1` lub `0` na podstawie `status`.


In [76]:
pomiary_new_col = pd.get_dummies(pomiary_time_index['status'])
pomiary_time_index = pd.concat([pomiary_time_index,pomiary_new_col], axis = 1)
pomiary_time_index.head()

,voltage,current,temperature,status,technician,broken,turned off,working
timestamp,,,,,,,,
2023-01-01 05:43:00,231.267702,1.245972,101.748627,broken,Emily Brown,True,False,False
2023-01-01 10:21:00,240.932880,1.527754,65.513886,working,Michael Johnson,False,False,True
2023-01-01 10:25:00,240.383591,1.778813,57.968485,working,Emily Brown,False,False,True
2023-01-01 12:28:00,0.000000,0.000000,54.056483,turned off,John Doe,False,True,False
2023-01-01 13:59:00,240.955130,1.225599,61.739875,working,John Doe,False,False,True


4. **Usuń wartości odstające (outliers) z kolumn `current` i `voltage`. Wartości odstające to te, które są większe niż 95. percentyl.**
   - *Wskazówka*: możesz użyć metody `.quantile()` do określenia granicy.


In [77]:
quant_95th_volt = pomiary_time_index['voltage'].quantile(0.95)
quant_95th_curr = pomiary_time_index['current'].quantile(0.95)
pomiary_quantiled = pomiary_time_index[(pomiary_time_index['voltage'] <= quant_95th_volt) & (pomiary_time_index['current'] <= quant_95th_curr)]
pomiary_quantiled.head()

,voltage,current,temperature,status,technician,broken,turned off,working
timestamp,,,,,,,,
2023-01-01 05:43:00,231.267702,1.245972,101.748627,broken,Emily Brown,True,False,False
2023-01-01 10:21:00,240.932880,1.527754,65.513886,working,Michael Johnson,False,False,True
2023-01-01 10:25:00,240.383591,1.778813,57.968485,working,Emily Brown,False,False,True
2023-01-01 12:28:00,0.000000,0.000000,54.056483,turned off,John Doe,False,True,False
2023-01-01 13:59:00,240.955130,1.225599,61.739875,working,John Doe,False,False,True


5. **Zastąp wartości odstające (outliers) w kolumnie `temperature` średnią temperaturą odpowiednią dla kategorii `status` (np. inną dla stanu `broken` i `working`).**
   - *Wskazówka*: Skorzystaj z grupowania przy pomocy `groupby()` na podstawie `status` i zastąp wartości odstające średnią z każdej grupy.


In [78]:
grouped_by_status = pomiary_quantiled.groupby('status').mean('temperature')
print(grouped_by_status.head())
print()


broken_mean_temp = grouped_by_status.loc['broken','temperature']
working_mean_temp = grouped_by_status.loc['working','temperature']
tunred_off_mean_temp = grouped_by_status.loc['turned off','temperature']
print('sprawdzam broken/temp')
print(broken_mean_temp)
print('sprawdzam working/temp')
print(working_mean_temp)
print('sprawdzam turned_off/temp')
print(tunred_off_mean_temp)

               voltage   current  temperature  broken  turned off  working
status                                                                    
broken      228.719639  1.566656   169.812762     1.0         0.0      0.0
turned off    0.000000  0.000000    63.516461     0.0         1.0      0.0
working     230.240442  1.593878   100.726006     0.0         0.0      1.0

sprawdzam broken/temp
169.81276231973985
sprawdzam working/temp
100.72600634081172
sprawdzam turned_off/temp
63.5164613070994


In [79]:
quant_95th_temp = pomiary_quantiled['temperature'].quantile(0.95)
pomiary_quantiled[(pomiary_quantiled['temperature'] > quant_95th_temp) & pomiary_quantiled['working'] == 1] = working_mean_temp
pomiary_quantiled[(pomiary_quantiled['temperature'] > quant_95th_temp) & pomiary_quantiled['turned off'] == 1] = tunred_off_mean_temp
pomiary_quantiled[(pomiary_quantiled['temperature'] > quant_95th_temp) & pomiary_quantiled['broken'] == 1] = broken_mean_temp


C:\Users\piotr\AppData\Local\Temp\ipykernel_29456\2294983761.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pomiary_quantiled[(pomiary_quantiled['temperature'] > quant_95th_temp) & pomiary_quantiled['working'] == 1] = working_mean_temp
C:\Users\piotr\AppData\Local\Temp\ipykernel_29456\2294983761.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '100.72600634081172' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  pomiary_quantiled[(pomiary_quantiled['temperature'] > quant_95th_temp) & pomiary_quantiled['working'] == 1] = working_mean_temp
C:\Users\piotr\AppData\Local\Temp\ipykernel_29456\2294983761.py:2: FutureWarning: S

6. **Dodaj dwie nowe kolumny: `power`, którą obliczysz na podstawie kolumn `voltage` i `current`, oraz `Normalized_Power`.**
   - *Wskazówka*: `power` to iloczyn `voltage` i `current`. Aby znormalizować `power`, użyj wzoru:  

     $\text{Normalized} = \frac{\text{Value} - \text{Min}}{\text{Max - Min}}$


In [80]:
pomiary_quantiled['power'] = pomiary_quantiled['current'] * pomiary_quantiled['voltage']
pomiary_quantiled['normalized_power'] = (pomiary_quantiled['power'] - pomiary_quantiled['power'].min()) / (pomiary_quantiled['power'].max() - pomiary_quantiled['power'].min())
pomiary_quantiled.head()


C:\Users\piotr\AppData\Local\Temp\ipykernel_29456\2323487512.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pomiary_quantiled['power'] = pomiary_quantiled['current'] * pomiary_quantiled['voltage']
C:\Users\piotr\AppData\Local\Temp\ipykernel_29456\2323487512.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pomiary_quantiled['normalized_power'] = (pomiary_quantiled['power'] - pomiary_quantiled['power'].min()) / (pomiary_quantiled['power'].max() - pomiary_quantiled['power'].min())


,voltage,current,temperature,status,technician,broken,turned off,working,power,normalized_power
timestamp,,,,,,,,,,
2023-01-01 05:43:00,231.267702,1.245972,101.748627,broken,Emily Brown,True,False,False,288.153058,0.009993
2023-01-01 10:21:00,240.932880,1.527754,65.513886,working,Michael Johnson,False,False,True,368.086250,0.012765
2023-01-01 10:25:00,240.383591,1.778813,57.968485,working,Emily Brown,False,False,True,427.597402,0.014828
2023-01-01 12:28:00,0.000000,0.000000,54.056483,turned off,John Doe,False,True,False,0.000000,0.000000
2023-01-01 13:59:00,240.955130,1.225599,61.739875,working,John Doe,False,False,True,295.314321,0.010241


7. **Standaryzuj wartości kolumny `temperature`.**
   - *Wskazówka*: Standaryzację można wykonać, odejmując średnią kolumny `temperature` i dzieląc przez jej odchylenie standardowe:  
     $ \text{Standardized} = \frac{\text{Value} - \text{Mean}}{\text{Std}} $
     użyj w tym celu metody `.apply` wraz z funcją `lambda`.


In [81]:
mean_temp = pomiary_quantiled['temperature'].mean()
std_temp = pomiary_quantiled['temperature'].std()
pomiary_quantiled['temperature_standardized'] = pomiary_quantiled['temperature'].apply([lambda x: (x-mean_temp)/std_temp])
print(pomiary_quantiled[['temperature', 'temperature_standardized']].head())

                     temperature  temperature_standardized
timestamp                                                 
2023-01-01 05:43:00   101.748627                  1.525817
2023-01-01 10:21:00    65.513886                 -0.212290
2023-01-01 10:25:00    57.968485                 -0.574228
2023-01-01 12:28:00    54.056483                 -0.761879
2023-01-01 13:59:00    61.739875                 -0.393322


C:\Users\piotr\AppData\Local\Temp\ipykernel_29456\1602007340.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pomiary_quantiled['temperature_standardized'] = pomiary_quantiled['temperature'].apply([lambda x: (x-mean_temp)/std_temp])


In [82]:
print('Std() dla standardized temp:')
print(pomiary_quantiled['temperature_standardized'].std())
print('Mean() dla standardized temp:')
print(pomiary_quantiled['temperature_standardized'].mean())

Std() dla standardized temp:
1.0
Mean() dla standardized temp:
1.763295392051719e-16


8. **Zmień nazwy wszystkich kolumn, aby zaczynały się z wielkiej litery.**
   - *Wskazówka*: Użyj `rename()` z funkcją `str.title()`.


In [83]:
pomiary_quantiled = pomiary_quantiled.rename(columns=lambda x: x.title())
print(pomiary_quantiled.columns.tolist())

['Voltage', 'Current', 'Temperature', 'Status', 'Technician', 'Broken', 'Turned Off', 'Working', 'Power', 'Normalized_Power', 'Temperature_Standardized']



9. **Dodaj 0,1 do każdego rekordu `Current`, który wystąpił przed 10 stycznia.**
   - *Wskazówka*: Użyj `loc[]` oraz warunku daty, aby zaktualizować tylko wybrane wartości `Current`.


In [84]:
pomiary_quantiled.loc[pomiary_quantiled.index < '2023-01-10', 'Current'] += 0.1

print(pomiary_quantiled.head())

                        Voltage   Current  Temperature      Status  \
timestamp                                                            
2023-01-01 05:43:00  231.267702  1.345972   101.748627      broken   
2023-01-01 10:21:00  240.932880  1.627754    65.513886     working   
2023-01-01 10:25:00  240.383591  1.878813    57.968485     working   
2023-01-01 12:28:00    0.000000  0.100000    54.056483  turned off   
2023-01-01 13:59:00  240.955130  1.325599    61.739875     working   

                          Technician Broken Turned Off Working       Power  \
timestamp                                                                    
2023-01-01 05:43:00      Emily Brown   True      False   False  288.153058   
2023-01-01 10:21:00  Michael Johnson  False      False    True  368.086250   
2023-01-01 10:25:00      Emily Brown  False      False    True  427.597402   
2023-01-01 12:28:00         John Doe  False       True   False    0.000000   
2023-01-01 13:59:00         John Doe  Fal